In [ ]:
!pip install fastapi nest-asyncio pyngrok uvicorn
!pip install -q transformers datasets evaluate 
!pip install python-multipart

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 16.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.5 MB/s eta 0:00:00
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=aaab2a3f970b0c7bfeb145fb9519c5bd3011f18a28e8e2788d174f55a140158c
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 63.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 56.2 MB/s eta 0:00:00
     ━━━━━━━━

In [ ]:
from transformers import SegformerFeatureExtractor, SegformerForSemanticSegmentation

hub_model_id = "bilal01/segformer-b0-finetuned-segments-stamp-verification"
feature_extractor = SegformerFeatureExtractor.from_pretrained("nvidia/segformer-b0-finetuned-ade-512-512")
model = SegformerForSemanticSegmentation.from_pretrained(f"{hub_model_id}")

/usr/local/lib/python3.10/dist-packages/transformers/models/segformer/feature_extraction_segformer.py:28: FutureWarning: The class SegformerFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use SegformerImageProcessor instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/segformer/image_processing_segformer.py:99: FutureWarning: The `reduce_labels` parameter is deprecated and will be removed in a future version. Please use `do_reduce_labels` instead.
  warnings.warn(


In [ ]:
from PIL import Image
from torch import nn
import torch
def process_image(filename): 
  # Load the image from file
  image = Image.open(filename)
  inputs = feature_extractor(images=image, return_tensors="pt")
  outputs = model(**inputs)
  logits = outputs.logits  # shape (batch_size, num_labels, height/4, width/4)

  # First, rescale logits to original image size
  upsampled_logits = nn.functional.interpolate(
      logits,
      size=image.size[::-1], # (height, width)
      mode='bilinear',
      align_corners=False
  )

  # Second, apply argmax on the class dimension
  pred_seg = upsampled_logits.argmax(dim=1)[0]
  # Check if any pixel in the mask is labeled as a stamp (value of 1)
  if torch.any(pred_seg == 1):
      return True
  else:
      return False

  # return pred_seg

In [ ]:
def stamp_test_pallete():
    return [
        [0, 0, 0],
        [216, 82, 24],
    ]

In [ ]:
import numpy as np

def get_seg_overlay(image, seg):
  color_seg = np.zeros((seg.shape[0], seg.shape[1], 3), dtype=np.uint8) # height, width, 3
  palette = np.array(stamp_test_pallete())
  for label, color in enumerate(palette):
      color_seg[seg == label, :] = color

  # Show image + mask
  img = np.array(image) * 0.5 + color_seg * 0.5
  img = img.astype(np.uint8)

  return img

In [ ]:
from fastapi import FastAPI, UploadFile, File
from fastapi.middleware.cors import CORSMiddleware
from starlette.responses import JSONResponse
from pydantic import BaseModel
import io
from fastapi.responses import StreamingResponse


app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.post('/index')
async def hello(file: UploadFile = File()):
    # Example: Save the image to a file
    with open('test.jpg', 'wb') as f:
        contents = await file.read()
        f.write(contents)

    image = Image.open("test.jpg")
    stamp_detected = process_image("test.jpg")
    # stamp_detected = get_seg_overlay(image, pred_seg)

    response = ''

    if stamp_detected:
      response = "Verified"
    else:
      response = "Not Verified"

    return { "response" : response}

    # predicted_img = Image.fromarray(predicted_img)
    # img_buffer = io.BytesIO()
    # predicted_img.save(img_buffer, format='JPEG')
    # img_buffer.seek(0)

    # Return the predicted_img as a StreamingResponse
    # return StreamingResponse(img_buffer, media_type='image/jpeg')

# @app.post('/index')
# async def hello(file: UploadFile = File()):
#     # Example: Save the image to a file
#     with open('test.jpg', 'wb') as f:
#         contents = await file.read()
#         f.write(contents)

#     image = Image.open("test.jpg")
#     pred_seg = process_image("test.jpg")
#     predicted_img = get_seg_overlay(image, pred_seg)

#     predicted_img = Image.fromarray(predicted_img)

#     # Save the predicted image to a file
#     output_file = "/content/static/predicted.jpg"
#     predicted_img.save(output_file, 'JPEG')

#     # Return the URL of the predicted image file
#     return {"url": output_file}




In [ ]:
# !mkdir -p /content/static/
# !chmod 777 /content/static/

In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Public URL: https://1dd0-35-223-238-116.ngrok.io


INFO:     Started server process [153]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     175.107.224.72:0 - "POST /index HTTP/1.1" 500 Internal Server Error


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/protocols/http/h11_impl.py", line 428, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/middleware/proxy_headers.py", line 78, in __call__
    return await self.app(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/fastapi/applications.py", line 276, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/starlette/applications.py", line 122, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/starlette/middleware/errors.py", line 184, in __call__
    raise exc
  File "/usr/local/lib/python3.10/dist-packages/starlette/middleware/errors.py", line 162, in __call__
    await self.app(scope, receive, _send)
  File "/usr/local/lib/python3.10/dist-packa

INFO:     175.107.224.72:0 - "POST /index HTTP/1.1" 500 Internal Server Error


...

INFO:     175.107.224.72:0 - "POST /index HTTP/1.1" 500 Internal Server Error


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/protocols/http/h11_impl.py", line 428, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/middleware/proxy_headers.py", line 78, in __call__
    return await self.app(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/fastapi/applications.py", line 276, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/starlette/applications.py", line 122, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/starlette/middleware/errors.py", line 184, in __call__
    raise exc
  File "/usr/local/lib/python3.10/dist-packages/starlette/middleware/errors.py", line 162, in __call__
    await self.app(scope, receive, _send)
  File "/usr/local/lib/python3.10/dist-packa

INFO:     175.107.224.72:0 - "POST /index HTTP/1.1" 500 Internal Server Error


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/protocols/http/h11_impl.py", line 428, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/middleware/proxy_headers.py", line 78, in __call__
    return await self.app(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/fastapi/applications.py", line 276, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/starlette/applications.py", line 122, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/starlette/middleware/errors.py", line 184, in __call__
    raise exc
  File "/usr/local/lib/python3.10/dist-packages/starlette/middleware/errors.py", line 162, in __call__
    await self.app(scope, receive, _send)
  File "/usr/local/lib/python3.10/dist-packa

INFO:     175.107.224.72:0 - "POST /index HTTP/1.1" 200 OK
INFO:     175.107.224.72:0 - "POST /index HTTP/1.1" 200 OK
INFO:     175.107.224.72:0 - "POST /index HTTP/1.1" 200 OK
INFO:     175.107.224.72:0 - "POST /index HTTP/1.1" 200 OK
INFO:     175.107.224.72:0 - "POST /index HTTP/1.1" 200 OK
INFO:     175.107.224.72:0 - "POST /index HTTP/1.1" 200 OK
INFO:     175.107.224.72:0 - "POST /index HTTP/1.1" 200 OK
INFO:     175.107.224.72:0 - "POST /index HTTP/1.1" 200 OK
INFO:     175.107.224.72:0 - "POST /index HTTP/1.1" 200 OK
INFO:     175.107.224.72:0 - "POST /index HTTP/1.1" 200 OK
INFO:     175.107.224.72:0 - "POST /index HTTP/1.1" 200 OK
